In [0]:
import sys 
!{sys.executable} -m pip install tensorflow

In [0]:
import sys 
!{sys.executable} -m pip install opencv-python


In [0]:
import os
import numpy as np
import cv2
import keras.backend as K
import random
import tensorflow as tf


def load_image(Path = './valid/XR_ELBOW', size = 512):
    Images = []
    for path in Path:
        try:
            image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,(size,size))
            Images.append(image)

        except Exception as e:
            print(str(e))

    Images = np.asarray(Images).astype('float32')
    Images=Images.reshape((len(Path),size,size,1))

    mean = np.mean(Images)			#normalization
    std = np.std(Images)
    Images = (Images - mean) / std
    return Images

In [4]:
def load_path(root_path = './valid/XR_ELBOW', size = 512):
	'''
	load MURA dataset
	'''

	Path = []
	labels = []
	for root,dirs,files in os.walk(root_path): #Read all pictures, os.walk Return to iterator genertor Traverse all files
		for name in files:
			path_1 = os.path.join(root,name)
			Path.append(path_1)
			if root.split('_')[-1]=='positive':	 #positive Label is 1，otherwise 0；
				labels+=[1]   	          	 #Last level directory file patient11880\\study1_negative\\image3.png
			else:
			    labels+=[0]
	print (len(Path))
	labels = np.asarray(labels)
	return Path, labels


Path,labels=load_path(r'/content/MURA-v1.1/train/XR_ELBOW',64)
images=load_image(Path,64)

4931


In [0]:
  def train_model():
        model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu,input_shape=(64,64,1)),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(128,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=1),#no padding strides is one
        tf.keras.layers.Dropout(0.25),       
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(2,activation=tf.nn.softmax)
        ])
        model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    
        model.fit(images,labels,epochs=25,verbose=2)
     
        return model

Batch Normalisation


AVerage Pooling

In [7]:
model=train_model()

W0801 12:09:32.534566 139960892516224 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/25
4931/4931 - 11s - loss: 0.7445 - acc: 0.5889
Epoch 2/25
4931/4931 - 9s - loss: 0.6528 - acc: 0.6106
Epoch 3/25
4931/4931 - 9s - loss: 0.6496 - acc: 0.6181
Epoch 4/25
4931/4931 - 9s - loss: 0.6288 - acc: 0.6449
Epoch 5/25
4931/4931 - 9s - loss: 0.6116 - acc: 0.6623
Epoch 6/25
4931/4931 - 9s - loss: 0.5857 - acc: 0.6846
Epoch 7/25
4931/4931 - 9s - loss: 0.5489 - acc: 0.7220
Epoch 8/25
4931/4931 - 9s - loss: 0.4961 - acc: 0.7589
Epoch 9/25
4931/4931 - 9s - loss: 0.4411 - acc: 0.7842
Epoch 10/25
4931/4931 - 9s - loss: 0.3682 - acc: 0.8335
Epoch 11/25
4931/4931 - 9s - loss: 0.3080 - acc: 0.8643
Epoch 12/25
4931/4931 - 9s - loss: 0.2497 - acc: 0.8925
Epoch 13/25
4931/4931 - 9s - loss: 0.2070 - acc: 0.9211
Epoch 14/25
4931/4931 - 9s - loss: 0.1770 - acc: 0.9341
Epoch 15/25
4931/4931 - 9s - loss: 0.1338 - acc: 0.9525
Epoch 16/25
4931/4931 - 9s - loss: 0.1222 - acc: 0.9574
Epoch 17/25
4931/4931 - 9s - loss: 0.1094 - acc: 0.9600
Epoch 18/25
4931/4931 - 9s - loss: 0.0743 - acc: 0.9747


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 63, 63, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        7

In [9]:
#Validation of test data
ValidPath,Validlabels=load_path(r'/content/MURA-v1.1/valid/XR_ELBOW',64)
Validimages=load_image(ValidPath,64)

465


Graphviz

In [0]:
score = model.evaluate(Validimages, Validlabels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
model.save('elbow.h5')